In [68]:
# PRODUCTION

import ipywidgets as widgets
from IPython.display import display, Markdown
from dotenv import load_dotenv
import os
import pdfplumber
from dotenv import load_dotenv

import PyPDF2
import fitz 
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import openai


load_dotenv()

True

In [69]:
# PRODUCTION
model_gpt = "gpt-4o-mini"

client = openai.OpenAI(
   
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Prepare the Document
### Extract Text from the PDF:

Install a library like PyPDF2 or pdfplumber to extract text from the PDF. I do it both


In [70]:
# PRODUCTION

file_path =r"C:\Users\Kholod\OneDrive\Desktop\AI_project_RAG\Labor Law.pdf" 
if not os.path.isfile(file_path):
    print(f"File not found: {file_path}")
else:
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:  
                text += page_text

    print(text)

Labor Law
Royal Decree No. M/51
September 27, 2005NOTE:
The translation of Saudi laws takes the following into consideration:
• Words used in the singular form include the plural and vice versa.
• Words used in the masculine form include the feminine.
• Words used in the present tense include the present as well as the future.
• The word “person” or “persons” and their related pronouns (he, his, him, they, their, them) refer to a natural and
legal person.
Page 2Labor Law
Part 1: Definitions and General Provisions
Chapter 1: Definitions
Article 1
This law shall be called the Labor Law.
Article 2
In this Law, the following terms shall have the meanings assigned thereto, unless the context requires otherwise:
Ministry: Ministry of Labor.
Minister: Minister of Labor.
Labor Office: The administrative authority assuming jurisdiction over the labor affairs within an area specified by a decision
of the Minister.
Employer: Any natural or corporate person employing one or more workers for a wage

In [71]:
# MAYBE PRODUCTION -- STORE PROCESSED TEXT
with open("output.txt", "w", encoding="utf-8") as output_file:
    output_file.write(text)


### Document Processing & Chunking

Break the text into manageable chunks (~500 tokens) for embedding

In [72]:
# PRODUCTION LOAD PREPROCESSED AND CHUNK

from langchain.text_splitter import RecursiveCharacterTextSplitter

### load the document that was stored before
def load_document(file_path):
    """Read the document content."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()
    


def split_into_chunks(text):
   
    if not text:
        print("No text extracted from the PDF.")
        return []
    else:
        # Initialize the text splitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=700,
            chunk_overlap=150,
            length_function=len
        )

        # Split the text into smaller chunks
        chunks = text_splitter.split_text(text)

        # Output the number of chunks and preview a few
        print(f"Generated {len(chunks)} chunks.")
        for i, chunk in enumerate(chunks[:5]):
            print(f"Chunk {i + 1}:\n{chunk}\n")
    return chunks
    



In [73]:
text = load_document("output.txt")
chunks = split_into_chunks(text)


Generated 191 chunks.
Chunk 1:
Labor Law
Royal Decree No. M/51
September 27, 2005NOTE:
The translation of Saudi laws takes the following into consideration:
• Words used in the singular form include the plural and vice versa.
• Words used in the masculine form include the feminine.
• Words used in the present tense include the present as well as the future.
• The word “person” or “persons” and their related pronouns (he, his, him, they, their, them) refer to a natural and
legal person.
Page 2Labor Law
Part 1: Definitions and General Provisions
Chapter 1: Definitions
Article 1
This law shall be called the Labor Law.
Article 2

Chunk 2:
Page 2Labor Law
Part 1: Definitions and General Provisions
Chapter 1: Definitions
Article 1
This law shall be called the Labor Law.
Article 2
In this Law, the following terms shall have the meanings assigned thereto, unless the context requires otherwise:
Ministry: Ministry of Labor.
Minister: Minister of Labor.
Labor Office: The administrative authority 

In [74]:
chunks

['Labor Law\nRoyal Decree No. M/51\nSeptember 27, 2005NOTE:\nThe translation of Saudi laws takes the following into consideration:\n• Words used in the singular form include the plural and vice versa.\n• Words used in the masculine form include the feminine.\n• Words used in the present tense include the present as well as the future.\n• The word “person” or “persons” and their related pronouns (he, his, him, they, their, them) refer to a natural and\nlegal person.\nPage 2Labor Law\nPart 1: Definitions and General Provisions\nChapter 1: Definitions\nArticle 1\nThis law shall be called the Labor Law.\nArticle 2',
 'Page 2Labor Law\nPart 1: Definitions and General Provisions\nChapter 1: Definitions\nArticle 1\nThis law shall be called the Labor Law.\nArticle 2\nIn this Law, the following terms shall have the meanings assigned thereto, unless the context requires otherwise:\nMinistry: Ministry of Labor.\nMinister: Minister of Labor.\nLabor Office: The administrative authority assuming jur

# 4. EMBED AND RETRIEVE

In [75]:
# PRODUCTION
#Store Embeddings in Vector Store
def store_in_vectorstore(chunks, embeddings, vectorstore_path="faiss_index"):
    """Stores embeddings in a FAISS vector store."""
    vectorstore = FAISS.from_texts(texts=chunks, embedding=embeddings)
    vectorstore.save_local(vectorstore_path)
    return vectorstore


In [76]:
bge_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")

In [77]:
vector_store = store_in_vectorstore(chunks, embeddings=bge_embeddings)

In [78]:
vector_store.similarity_search("Labor Law")

[Document(metadata={}, page_content='employer shall be required to:\n1. Refrain from resorting to forced labor, or withholding the worker’s wage or part thereof without court order, or\nmistreating the worker in any manner that may infringe upon his dignity or religion.\n2. Give the workers the time required to exercise their rights as provided for in this Law without any deductions from\ntheir wages against such time. He may regulate the exercise of this right in a manner not detrimental to work progress.\n3. Facilitate for the employees of the competent authorities any task related to the enforcement of the provisions of this\nLaw.\nArticle 62'),
 Document(metadata={}, page_content='1. Access any firm that is subject to the provisions of the Labor Law at any time, day or night, without prior notice.\n2. Perform any examination or investigation required to ascertain the proper implementation of the Law. They may in\nparticular:\na) Question the employer, his representative, or the wor

In [79]:
#loading vector store from hard drive to RAM
vector_store = FAISS.load_local("faiss_index", embeddings=bge_embeddings, allow_dangerous_deserialization=True)

# Generation

Feed retrieved docs to OpenAI API with question. do prompt engineering

In [80]:
def generate_response_with_history(user_input: str, context: str, model: str = "gpt-3.5-turbo") -> str:
    """
    Generate a response from OpenAI using the chat history and user input.

    Args:
        user_input (str): The latest user query.
        context (str): The system context or additional information.
        model (str): The OpenAI model to use.

    Returns:
        str: The assistant's response.
    """
    # Build the message history with context and previous messages
    messages = [{"role": "system", "content": context}]
    messages.extend(st.session_state.messages)  # Include previous messages
    messages.append({"role": "user", "content": user_input})  # Add the current user input

    # Generate response using OpenAI API
    response = openai.ChatCompletion.create(
        model=model,  # Use "gpt-3.5-turbo" or "gpt-4"
        messages=messages
    )

    # Extract the assistant's message from the API response
    assistant_message = response["choices"][0]["message"]["content"]

    # Save the current user input and assistant's response to the chat history
    st.session_state.messages.append({"role": "user", "content": user_input})
    st.session_state.messages.append({"role": "assistant", "content": assistant_message})

    return assistant_message


In [83]:
retrieved_chunks[0].page_content

'Article 41\nThe Regulations shall specify the conditions, controls, and procedures for recruitment from abroad, transfer of services, and\nchange of profession.\nPART 4: Training and Qualification\nChapter 1: Training and Qualification of the Employer’s Workers\nArticle 42\nAn employer shall be required to prepare his Saudi workers and enhance their technical, administrative, vocational, and\nother skills for the purpose of gradually replacing non-Saudis.\nThe employer shall keep a record showing the names of the Saudi workers who have replaced non-Saudis in accordance with\nthe conditions and rules set forth in the Regulations.\nArticle 43'

In [84]:
import ipywidgets as widgets
from IPython.display import display, Markdown
# Just a visualization function to make it easier to interact with the API
def create_ui(process_function):
    """Create a simple UI that works with any processing function."""
    # Create widgets
    input_box = widgets.Textarea(
        description='Input:',
        layout=widgets.Layout(width='600px', height='100px')
    )
    submit_button = widgets.Button(description='Submit')
    output_area = widgets.Output()
    response_area = widgets.HTML(value='<h3>Chat</h3>')
    def on_submit(b):
        with output_area:
            output_area.clear_output()
            response = process_function(input_box.value)
            response = (
                "**Question:** " + input_box.value + "\n\n**Answer:**\n" + response
            )
            input_box.value = ""  # Clear input box after submission
            display(Markdown(response))  # Display formatted response
    submit_button.on_click(on_submit)
    # Layout
    ui = widgets.VBox([
        input_box,
        submit_button,
        response_area,
        output_area
    ])
    display(ui)

In [48]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Create UI components
query_input = widgets.Textarea(
    description='Query:',
    layout=widgets.Layout(width='600px', height='100px'),
    placeholder="Enter your query here..."
)

submit_button = widgets.Button(description='Submit')
output_area = widgets.Output()

# Define the function to handle query submission
def handle_query_submission(_):
    with output_area:
        output_area.clear_output()
        query = query_input.value
        retrieved_chunks = vector_store.similarity_search(query, k=5)
        response = generate_response_with_openai(retrieved_chunks, query)
        display(Markdown(f"**Query:** {query}\n\n**Response:**\n{response}"))

submit_button.on_click(handle_query_submission)

# Arrange the UI layout
ui_layout = widgets.VBox([query_input, submit_button, output_area])
display(ui_layout)
